In [1]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
import math
from torch.utils.data import Dataset, DataLoader
from sklearn import datasets
from torchvision import transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.preprocessing import StandardScaler #para escalar caracteristicas
from sklearn.model_selection import train_test_split # separar mas facil la data de train y test

# Dataset and Dataloader
* En grandes cantidades de datos es mejor splitearlos en lotes o "batches", esto implica un doble bucle
* Desde ahora tener en cuenta
    * epoch = 1 forwad y backward en TODO los training samples
    * batch_size = numero de training samples en 1 forward y backward pass
    * num iteraciones = numero de passes, cada pass usando [batch_size] numero de samples
    * por ej 100 samples, 20 batch_size = 100/5 iteraciones x 1 epoch
    
* Intentaremos predecir la categoria del vino en wine.csv (categorias 1,2,3)
* Se puede representar el Dataset como clase de esta forma

In [2]:
class WineDataset(Dataset): #Hereda Dataset
    def __init__(self):
        #data loading
        xy = np.loadtxt('wine.csv', delimiter=",", dtype=np.float32, skiprows=1)
        self.x = torch.from_numpy(xy[:, 1:]) # queremos todas las muestras y todas columnas menos la primera
        self.y = torch.from_numpy(xy[:,[0]]) #  queremos todas las muestras pero solo la primera columna
        # ambas representarlas como tensor
        self.n_samples = xy.shape[0] #
        
        
    def __getitem__(self, index):
        # dataset[0]
        return self.x[index], self.y[index]
        
    def __len__(self):
        # len(dataset)
        return self.n_samples
        
dataset = WineDataset()

firts_data = dataset[0]
features, labels = firts_data

print(features,labels)


tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]) tensor([1.])


tambien se puede cargar data

In [3]:
dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=True)#, num_workers=1)
#Shuffle es muy util para el training pues es para mezclar lsos datos
#num workers puede acelerar el proceso dependiendo el numero de procesadores, a veces no funciona

datatiter = iter(dataloader)
data = datatiter.next()
features, labels = data
print(features, labels)





tensor([[1.3320e+01, 3.2400e+00, 2.3800e+00, 2.1500e+01, 9.2000e+01, 1.9300e+00,
         7.6000e-01, 4.5000e-01, 1.2500e+00, 8.4200e+00, 5.5000e-01, 1.6200e+00,
         6.5000e+02],
        [1.4220e+01, 3.9900e+00, 2.5100e+00, 1.3200e+01, 1.2800e+02, 3.0000e+00,
         3.0400e+00, 2.0000e-01, 2.0800e+00, 5.1000e+00, 8.9000e-01, 3.5300e+00,
         7.6000e+02],
        [1.2250e+01, 1.7300e+00, 2.1200e+00, 1.9000e+01, 8.0000e+01, 1.6500e+00,
         2.0300e+00, 3.7000e-01, 1.6300e+00, 3.4000e+00, 1.0000e+00, 3.1700e+00,
         5.1000e+02],
        [1.2850e+01, 3.2700e+00, 2.5800e+00, 2.2000e+01, 1.0600e+02, 1.6500e+00,
         6.0000e-01, 6.0000e-01, 9.6000e-01, 5.5800e+00, 8.7000e-01, 2.1100e+00,
         5.7000e+02]]) tensor([[3.],
        [1.],
        [2.],
        [3.]])


In [6]:
# training loop
num_epoch = 2
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/4)

for epoch in range(num_epoch):
    # segundo loop con los lotes
    for i,(inputs, labels) in enumerate(dataloader):
        # forward, backward, update... esto es un ejemplo dummy asi q no se ara
        if (i+1) % 5 == 0:
            print(f'epoch {epoch+1}/{num_epoch}, step {i+1}/{n_iterations}, inputs {inputs.shape}')

epoch 1/2, step 5/45, inputs torch.Size([4, 13])
epoch 1/2, step 10/45, inputs torch.Size([4, 13])
epoch 1/2, step 15/45, inputs torch.Size([4, 13])
epoch 1/2, step 20/45, inputs torch.Size([4, 13])
epoch 1/2, step 25/45, inputs torch.Size([4, 13])
epoch 1/2, step 30/45, inputs torch.Size([4, 13])
epoch 1/2, step 35/45, inputs torch.Size([4, 13])
epoch 1/2, step 40/45, inputs torch.Size([4, 13])
epoch 1/2, step 45/45, inputs torch.Size([2, 13])
epoch 2/2, step 5/45, inputs torch.Size([4, 13])
epoch 2/2, step 10/45, inputs torch.Size([4, 13])
epoch 2/2, step 15/45, inputs torch.Size([4, 13])
epoch 2/2, step 20/45, inputs torch.Size([4, 13])
epoch 2/2, step 25/45, inputs torch.Size([4, 13])
epoch 2/2, step 30/45, inputs torch.Size([4, 13])
epoch 2/2, step 35/45, inputs torch.Size([4, 13])
epoch 2/2, step 40/45, inputs torch.Size([4, 13])
epoch 2/2, step 45/45, inputs torch.Size([2, 13])


Se ve claramente las epocas, steps y q los lotes son de 4 x 13
De esta forma de formatea la cantidad correcta de como manejar los datos